In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential 
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
num_classes=5
img_rows,img_cols=48,48
batch_size=32

In [3]:
train_data_dir=r'C:\Users\Dell\Downloads\Emotion Dataset\fer2013\train'
validation_data_dir=r'C:\Users\Dell\Downloads\Emotion Dataset\fer2013\validation'


In [4]:
train_datagen=ImageDataGenerator(
          rescale=1./255,
          rotation_range=30,
          shear_range=0.3,
          zoom_range=0.3,
          width_shift_range=0.4,
          height_shift_range=0.4,
          horizontal_flip=True,
          fill_mode='nearest')
validation_datagen=ImageDataGenerator(rescale=1./255)


In [5]:
train_generator=train_datagen.flow_from_directory(
          train_data_dir,
          color_mode='grayscale',
          target_size=(img_rows,img_cols),
          batch_size=batch_size,
          class_mode='categorical',
          shuffle=True)
validation_generator=validation_datagen.flow_from_directory(
          validation_data_dir,
          color_mode='grayscale',
          target_size=(img_rows,img_cols),
          batch_size=batch_size,
          class_mode='categorical',
          shuffle=True)

Found 24176 images belonging to 5 classes.
Found 3006 images belonging to 5 classes.


In [6]:
model=Sequential()

#BLOCK-1#
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#BLOCK-2#
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#BLOCK-3#
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#BLOCK-4#
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#BLOCK-5#
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#BLOCK-6#
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#BLOCK-7#
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [7]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)       

In [16]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint=ModelCheckpoint('Emotion_Recognition.h5',
                          monitor='val_loss',
                          mode='min',
                          save_best_only='True',
                          verbose=1)
earlystop=EarlyStopping(monitor='val_loss',
                       min_delta=0,
                       patience=9,
                       verbose=1,
                       restore_best_weights=True)
reduce_lr=ReduceLROnPlateau(monitor='val_loss',
                           factor=0.2,
                           patience=3,
                           verbose=1,
                           min_delta=0.0001)
callbacks=[earlystop,checkpoint,reduce_lr]

In [17]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.001),
             metrics=['accuracy'])
train_samples=24176
validation_samples=3006

history=model.fit_generator(
     train_generator,
     steps_per_epoch=train_samples//batch_size,
     epochs=25,
     callbacks=callbacks,
     validation_data=validation_generator,
     validation_steps=validation_samples//batch_size)

Epoch 1/25
755/755 [==============================] - 560s 741ms/step - loss: 1.8563 - accuracy: 0.2400 - val_loss: 1.6411 - val_accuracy: 0.3004

Epoch 00001: val_loss improved from inf to 1.64110, saving model to Emotion_Recognition.h5
Epoch 2/25
755/755 [==============================] - 367s 487ms/step - loss: 1.5785 - accuracy: 0.2840 - val_loss: 1.6492 - val_accuracy: 0.3073

Epoch 00002: val_loss did not improve from 1.64110
Epoch 3/25
755/755 [==============================] - 368s 488ms/step - loss: 1.5511 - accuracy: 0.3004 - val_loss: 1.5201 - val_accuracy: 0.3178

Epoch 00003: val_loss improved from 1.64110 to 1.52008, saving model to Emotion_Recognition.h5
Epoch 4/25
755/755 [==============================] - 370s 489ms/step - loss: 1.5356 - accuracy: 0.3111 - val_loss: 1.7280 - val_accuracy: 0.2976

Epoch 00004: val_loss did not improve from 1.52008
Epoch 5/25
755/755 [==============================] - 386s 512ms/step - loss: 1.4902 - accuracy: 0.3369 - val_loss: 1.2664 -